In [147]:
import numpy as np
import pandas as pd
train_df= pd.read_csv('titanic/train.csv')
test_df= pd.read_csv('titanic/test.csv')

def clean(df):
    df['Age'].fillna(train_df["Age"].median(skipna=True), inplace= True)
    df["Embarked"].fillna(train_df["Embarked"].value_counts().idxmax(), inplace=True)
    df.drop("Cabin", axis=1, inplace=True)
    df.drop("Name", axis=1, inplace=True)
    df.drop("Ticket", axis=1, inplace=True)
    df.drop("PassengerId",axis=1,inplace=True)
    
    return df
train= clean(train_df)
test= clean(test_df)

li= [round(num) for num in train['Fare']]
train['Fare']=li

li= [round(num) for num in train['Age']]
train['Age']=li

train['Sex'] =train['Sex'].map({'male':1,'female':0})
train['Embarked'] =train['Embarked'].map({'S':0,'C':1,'Q':2})

#-------------------------
test= test.fillna(train['Fare'].median())

lu= [round(num) for num in test['Fare']]
test['Fare']=lu

lu= [round(num) for num in test['Age']]
test['Age']=lu

test['Sex'] =test['Sex'].map({'male':1,'female':0})
test['Embarked'] =test['Embarked'].map({'S':0,'C':1,'Q':2})

from sklearn.model_selection import train_test_split
X= train.drop('Survived',axis=1).to_numpy()
Y= pd.DataFrame(train['Survived']).to_numpy()

In [5]:
X.shape

(891, 7)

In [165]:
from sklearn.metrics import mean_squared_error

In [161]:
def initialize_parameter(n_x,n_h,n_y):
    W1= np.random.randn(n_h,n_x) *0.01
    b1= np.zeros((n_h,1))
    
    W2= np.random.randn(n_y,n_h)*0.01
    b2= np.zeros((n_y,1))
    
    parameters={
        'W1':W1,
        'b1':b1,
        'W2':W2,
        'b2':b2
    }
    return parameters

parameters= initialize_parameter(n_x,n_h,n_y)



In [169]:
def relu(x):
    return np.maximum(0.1*x, x)
def sigmoid(z):
    return 1/ (1+ np.exp(-z))

def gradient_descent(X,Y,parameters,learning_rate,num_iteration):
    m= len(Y)
    W1= parameters['W1']
    W2= parameters['W2']
    b1= parameters['b1']
    b2= parameters['b2']
    
    for i in range(num_iteration):
        Z1= np.dot(W1,X.T) + b1
        A1= relu(Z1)
        Z2= np.dot(W2,A1) + b2
        A2= sigmoid(Z2)
        
        cost= -(1/m) * np.sum( np.dot(Y.T,np.log(A2.T)) + np.dot((1-Y.T),np.log(1-A2.T)))
        if i%5 ==0:
            print(f'Iteration {i} : cost --> {cost*100}')

        dZ2= A2 -Y.T #(1, 891)
        
        dW2= np.dot(dZ2,A1.T) * (1/m)
        dB2= np.sum(dZ2,axis=1,keepdims=True) * (1/m)

    #     dz1--> da1--> dz2-->da2
        def G_prime(z):
            z= pd.DataFrame(z)<0
            for col in z.columns:
                z[col]= z[col].map({True:0.01,False:1}) 
            return z.to_numpy()

        dA2=  np.dot(W2.T,Z2)
        G_prime= G_prime(Z1)

        dZ1= dA2 * G_prime    
        dW1= np.dot(dZ1,X)* (1/m)
        dB1= np.sum(dZ1,axis=1,keepdims=True)* (1/m)
        
        W1 = W1 - learning_rate * dW1
        W2 = W2 - learning_rate * dW2
        b1 = b1 - learning_rate * dB1
        b2 = b2 - learning_rate * dB2
        
    parameters={
        'W1':W1,
        'W2':W2,
        'b1':b1,
        'b2':b2
    }
    return parameters
        
def one_hidden_layer(X,Y,parameters,learning_rate,num_iteration):

    
    parameters= gradient_descent(X,Y,parameters,learning_rate,num_iteration)
    return parameters
    
    
    
n_x= X.shape[1]
n_h= 4
n_y= 1


best_parameters= one_hidden_layer(X,Y,parameters,0.12,1000)



Iteration 0 : cost --> 69.29810421329478
Iteration 5 : cost --> 68.46214760640899
Iteration 10 : cost --> 68.20485370245275
Iteration 15 : cost --> 67.6293203330235
Iteration 20 : cost --> 67.35434420618877
Iteration 25 : cost --> 67.12939983921892
Iteration 30 : cost --> 66.97710790523826
Iteration 35 : cost --> 66.84713143478321
Iteration 40 : cost --> 66.74086867618546
Iteration 45 : cost --> 66.64913228590274
Iteration 50 : cost --> 66.56432702123038
Iteration 55 : cost --> 66.48000693377045
Iteration 60 : cost --> 66.39071524103075
Iteration 65 : cost --> 66.29214347132056
Iteration 70 : cost --> 66.18162198494939
Iteration 75 : cost --> 66.05883946908631
Iteration 80 : cost --> 65.92647558832105
Iteration 85 : cost --> 65.79023123563512
Iteration 90 : cost --> 65.65782070920648
Iteration 95 : cost --> 65.53703935250907
Iteration 100 : cost --> 65.43373691893595
Iteration 105 : cost --> 65.35069723439804
Iteration 110 : cost --> 65.28778624751945
Iteration 115 : cost --> 65.242920

KeyboardInterrupt: 

In [142]:
def predict(test,parameters):
    W1= parameters['W1']
    W2= parameters['W2']
    b1= parameters['b1']
    b2= parameters['b2']
    
    Z1= np.dot(W1,X.T) + b1
    A1= relu(Z1)
    Z2= np.dot(W2,A1) + b2
    A2= sigmoid(Z2)
    return A2
output= predict(test,best_parameters)
answer= pd.DataFrame(output.T)<0.5
answer[0].map({True:1,False:0})


0      1
1      1
2      1
3      1
4      1
      ..
886    1
887    1
888    1
889    1
890    1
Name: 0, Length: 891, dtype: int64

In [ ]:
test_df= pd.read_csv('titanic/test.csv')

In [ ]:
test_df.head()